In [15]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does not exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)



enter file name: Reading data from file 'lab1-cloetta.mat'
Datafile: 'lab1-cloetta.mat'
Rows: 3 cols: 5
c: [-2 -1  0  0  0]
b: [ 1.    0.75 10.  ]
A: [[ 1  1  1  0  0]
 [ 1  0  0  1  0]
 [ 8 20  0  0  1]]
bix: [2 3 4]
xcheat: [0.75 0.2  0.05 0.   0.  ]
zcheat: -1.7


In [54]:
import numpy as np
rc_min = np.amin(c)
inkvar = int(np.where(c == np.amin(c))[0]) #vi ska nog använda asanarray och flatten. kommer nog inte funka om vi har flera np.

inkKol = A[:, inkvar]
utgKol = b/inkKol
utgvar = int(np.where(utgKol == np.amin(utgKol))[0])


print(inkvar)
print(utgvar)




0
1


In [19]:
import numpy as np
import time
import sys

rc_min = np.nanmin(c)
print(rc_min)

# npzfile= fil
# c=npzfile['c']
# b=npzfile['b']
# A=npzfile['A']
# bix=npzfile['bix']
# zcheat=npzfile['zcheat']
# xcheat=npzfile['xcheat']


t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))

# Create nix
nix=np.setdiff1d(range(n), bix) 

B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]

# Start iterations
iter = 0

while iter >= 0:
    iter+=1
    
    # calc right-hand-sides and reduced costs
    
    # --------


    # calc most negative reduced cost, rc_min,
    # and index for entering variable, inkvar
    # --------
    rc_min = 1; 


    if rc_min >= -1.0E-12:
        print ('Ready')
        iter=-1
 
        # construct solution, x, and check it
        # --------

        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        # calc entering column, a
        # --------

        if max(a) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            # calc leaving var, utgvar
            # --------


            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # --------


elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

-2
Rows: 3 cols: 5
Ready


NameError: name 'x' is not defined